# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [10]:
import requests

from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

url = "https://books.toscrape.com/index.html"
response = requests.get(url)
response


<Response [200]>

In [11]:
response.headers


{'Date': 'Thu, 11 Sep 2025 18:55:47 GMT', 'Content-Type': 'text/html', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Last-Modified': 'Wed, 08 Feb 2023 21:02:32 GMT', 'ETag': 'W/"63e40de8-c85e"', 'Strict-Transport-Security': 'max-age=0; includeSubDomains; preload', 'Content-Encoding': 'br'}

In [12]:
print(response.headers['Content-Type'])

text/html


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin

def scrape_books(min_rating, max_price):
    """
    Scrapes book data from the "Books to Scrape" website based on minimum rating and maximum price.

    Args:
        min_rating (int): The minimum rating (1-5) of books to include.
        max_price (float): The maximum price of books to include.

    Returns:
        pandas.DataFrame: A DataFrame containing the scraped book data with columns:
                          'UPC', 'Title', 'Price (£)', 'Rating', 'Genre',
                          'Availability', and 'Description'.
    """
    base_url = "https://books.toscrape.com/catalogue/"
    current_url = urljoin(base_url, 'page-1.html')
    scraped_books = []
    
    # Map star ratings from text to integers
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    print("Starting the scraping process...")

    while current_url:
        try:
            response = requests.get(current_url)
            response.raise_for_status()  # Raise an exception for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find all book articles on the current page
            books_on_page = soup.find_all('article', class_='product_pod')

            for book in books_on_page:
                # Get the link to the book's detail page
                book_link = book.find('h3').find('a')['href']
                book_url = urljoin(base_url, book_link)

                # Scrape the individual book page
                try:
                    book_response = requests.get(book_url)
                    book_response.raise_for_status()
                    book_soup = BeautifulSoup(book_response.content, 'html.parser')

                    # --- Initial Extraction for Filtering ---
                    # Extract rating
                    rating_p = book_soup.find('p', class_='star-rating')
                    rating_text = rating_p['class'][1] if rating_p else 'Zero'
                    rating = rating_map.get(rating_text, 0)
                    
                    # Extract price
                    price_str = book_soup.find('p', class_='price_color').text.replace('£', '')
                    price = float(price_str)

                    # --- Filtering ---
                    if rating >= min_rating and price <= max_price:
                        # --- Detailed Extraction for Filtered Books ---
                        # Extract title
                        title = book_soup.find('h1').text

                        # Extract product information table data
                        table = book_soup.find('table', class_='table-striped')
                        rows = table.find_all('tr')
                        upc = rows[0].find('td').text
                        availability_str = rows[5].find('td').text
                        availability = 'In stock' in availability_str
                        
                        # Extract genre from breadcrumbs
                        genre = book_soup.find('ul', class_='breadcrumb').find_all('li')[2].find('a').text
                        
                        # Extract description
                        description_tag = book_soup.find('div', id='product_description')
                        description = description_tag.find_next_sibling('p').text if description_tag else "No description available."

                        scraped_books.append({
                            "UPC": upc,
                            "Title": title,
                            "Price (£)": price,
                            "Rating": rating,
                            "Genre": genre,
                            "Availability": "In Stock" if availability else "Out of Stock",
                            "Description": description
                        })
                
                except requests.exceptions.RequestException as e:
                    print(f"Could not process book URL {book_url}: {e}")
                except (AttributeError, KeyError, IndexError) as e:
                    print(f"Error parsing details for a book at {book_url}: {e}")

            # Find the next page link
            next_page_tag = soup.find('li', class_='next')
            if next_page_tag:
                next_page_link = next_page_tag.find('a')['href']
                current_url = urljoin(base_url, next_page_link)
                print(f"Navigating to next page: {current_url}")
            else:
                current_url = None # No more pages

        except requests.exceptions.RequestException as e:
            print(f"Could not retrieve page {current_url}: {e}")
            break
            
    print("Scraping process finished.")
    
    # Create a pandas DataFrame
    df = pd.DataFrame(scraped_books, columns=[
        "UPC", "Title", "Price (£)", "Rating", "Genre", 
        "Availability", "Description"
    ])
    
    return df

# --- Main execution block ---
if __name__ == '__main__':
    # Set the desired filter criteria
    MINIMUM_RATING = 4
    MAXIMUM_PRICE = 20.00
    
    # Execute the scraping function
    filtered_books_df = scrape_books(min_rating=MINIMUM_RATING, max_price=MAXIMUM_PRICE)
    
    # Display the results
    print(f"\nFound {len(filtered_books_df)} books with a rating of {MINIMUM_RATING}+ stars and price under £{MAXIMUM_PRICE:.2f}\n")
    
    if not filtered_books_df.empty:
        # To display all columns without truncation
        pd.set_option('display.max_columns', None)
        # To display the full description without truncation
        pd.set_option('display.max_colwidth', None)
        print(filtered_books_df)
    else:
        print("No books found matching the specified criteria.")



Starting the scraping process...
Navigating to next page: https://books.toscrape.com/catalogue/page-2.html
Navigating to next page: https://books.toscrape.com/catalogue/page-3.html
Navigating to next page: https://books.toscrape.com/catalogue/page-4.html
Navigating to next page: https://books.toscrape.com/catalogue/page-5.html
Navigating to next page: https://books.toscrape.com/catalogue/page-6.html
Navigating to next page: https://books.toscrape.com/catalogue/page-7.html
Navigating to next page: https://books.toscrape.com/catalogue/page-8.html
Navigating to next page: https://books.toscrape.com/catalogue/page-9.html
Navigating to next page: https://books.toscrape.com/catalogue/page-10.html
Navigating to next page: https://books.toscrape.com/catalogue/page-11.html
Navigating to next page: https://books.toscrape.com/catalogue/page-12.html
Navigating to next page: https://books.toscrape.com/catalogue/page-13.html
Navigating to next page: https://books.toscrape.com/catalogue/page-14.html
